Import of libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler # To standardize the data
import cvxpy as cp

Import of the HMLasso function

In [2]:
### Adapt the path "C:/Users/Kilian/Desktop/ENSAE/STATAPP" to run the cell

import sys
sys.path.insert(1, 'C:/Users/Kilian/Desktop/ENSAE/STATAPP/Projet_Statapp/pretreatment')

import file_04_HMLasso as hml

## Data downloading and separation of the dataset

Dataset containing the types of each column from data_03.csv

In [3]:
columns_types = pd.read_csv("data_03_columns_types.csv", index_col=0)
columns_types.head(3)

,Name,Type
0,HHIDPN,Cont
1,HHID,Char
2,PN,Char


Downloading the data with social and genetic variables.

In [4]:
data = pd.read_csv("data_03.csv")

C:\Users\Kilian\AppData\Local\Temp\ipykernel_3292\3952530018.py:1: DtypeWarning: Columns (2684) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data_03.csv")


The column "genetic_Section_A_or_E" have mixed types, so we change its format.

In [5]:
temporary = np.where(data['genetic_Section_A_or_E'] == 'E', 1, np.where(data['genetic_Section_A_or_E'] == 'A', 0, np.nan))

In [6]:
data["genetic_Section_A_or_E"] = temporary

Now we add the health index created by t-SNE

In [7]:
tSNE_GHI = pd.read_csv("data_tSNE_GHI.csv")

We merge the t-SNE health index to the data

In [8]:
data = data.merge(tSNE_GHI, how ='left', on ='HHIDPN')

The final outcome to predict is tSNE_GHI14, so we only keep individuals who were interviewed during the last wave (14th wave)

In [9]:
data_bis = data[data['tSNE_GHI14'].notna()]

Number of individuals present in every waves.

In [10]:
tSNE_GHI[~tSNE_GHI.isnull().any(axis=1)].shape[0]

3396

We select the outcome tSNE_GHI

In [11]:
Y = data_bis[["HHIDPN"]+["tSNE_GHI" + str(i) for i in range (1,15)]]

We drop the previous health index GHIw from the data, which won't be used as outcome.
(list_columns_GHI contains the names of GHIw columns).

We drop the outcome to create the matrix X.

In [12]:
X = data_bis.drop(["GHI" + str(i) for i in range (1,15)], axis = 1)
X.drop(["tSNE_GHI" + str(i) for i in range (1,15)], axis = 1, inplace =True)

Now we split the dataset into training, validation and test sets.

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=18)
X_test, X_valid, Y_test, Y_valid = train_test_split(X_test, Y_test, test_size=0.5, random_state = 6)

Smaller sets while coding

In [14]:
nb_test, nb_train, nb_valid = len(X_test.index)//10, len(X_train.index)//10, len(X_valid.index)//10
X_test, Y_test = X_test.iloc[:nb_test], Y_test.iloc[:nb_test]
X_train, Y_train = X_train.iloc[:nb_train], Y_train.iloc[:nb_train]
X_valid, Y_valid = X_valid.iloc[:nb_valid], Y_valid.iloc[:nb_valid]

## Machine learning

The objective here is to make a dataset where we observe if each variable exists at each wave

In [15]:
def dataset_temporal_variables (X_train,add_tSNE_GHIw):   
    temporal_variables = {}
    waves_columns = [col for col in X_train.columns if "genetic_" not in col and col[1] in "123456789"]
    for col in waves_columns:
      char = col[0] # R or H
      if col[2] in "01234":
        wave = col[1:3]
        suffix = col[3:]
      else:
        wave = col[1]
        suffix = col[2:]
      variable = char + 'w' + suffix

      if variable not in temporal_variables.keys():
        temporal_variables[variable] = np.zeros((14), dtype=bool)

      temporal_variables[variable][int(wave)-1] = True

    temporal_variables = pd.DataFrame(temporal_variables)

    # We manually add "tSNE_GHIw":
    if add_tSNE_GHIw:
        temporal_variables["tSNE_GHIw"] = np.ones((14), dtype=bool)
        waves_columns += [f"tSNE_GHI{w}" for w in range(1,15)]
        
    return (temporal_variables,waves_columns)

In [16]:
# Timeless data
def timeless_variables(X_train,waves_columns):
    non_waves_columns = [col for col in X_train.columns if col not in waves_columns]
    To_remove = ["HHIDPN","PN","HHID","RAHHIDPN"]+["INW"+str(i+1) for i in range (14)]
    for x in To_remove:
        non_waves_columns.remove(x)
    return non_waves_columns

We put the explaining variables by wave in a list of dataset Intemporal variables are put in each one of them

In [17]:
import random

def list_wave(X_train, reduced):
    (temporal_variables , waves_columns) = dataset_temporal_variables(X_train,True)
    non_waves_columns = timeless_variables(X_train,waves_columns)
    
    #Reduce number of variables to code
    if reduced:
        temporal_variables_2 = temporal_variables.iloc[:,[i for i in range(1,15)]+[-i for i in range(1,5)]]
        non_waves_columns_2 = random.choices(non_waves_columns,k=5)

        liste = [] 
        for i in range(14):
            columns_wave_i = ["HHIDPN"]+[col.replace('w', str(i+1)) for col in temporal_variables_2.T[i].index[temporal_variables_2.T[i]] if col != "tSNE_GHIw"]
            #Add the intemporal variables only to the last wave, to avoid duplicated labels issues
            if i == 13:
                liste.append(X_train.loc[X_train["INW"+str(i+1)] == 1, columns_wave_i + non_waves_columns_2])
            else:
                liste.append(X_train.loc[X_train["INW"+str(i+1)] == 1, columns_wave_i])
                
    #All the variables
    else:
        liste = []    # len = 14 
        for i in range(14):
            columns_wave_i = ["HHIDPN"]+[col.replace('w', str(i+1)) for col in temporal_variables.T[i].index[temporal_variables.T[i]] if col != "tSNE_GHIw"]
            #Intemporal variables only in the first wave, to avoid duplicated labels issues
            if i ==  13:
                liste.append(X_train.loc[X_train["INW"+str(i+1)] == 1, columns_wave_i + non_waves_columns])
            else:
                liste.append(X_train.loc[X_train["INW"+str(i+1)] == 1, columns_wave_i])
                
    return (liste)
    

### HMLasso selection

We start to initialize with a first lasso on the first wave.

In [18]:
def initialize_lasso(liste, Y_train, mu, limit):
    
    print("wave",1)
    
    scaler = StandardScaler()#(with_std=False)
    hml.ERRORS_HANDLING = "ignore"
    
    #Prepare data
    X_train1 = liste[0].drop("HHIDPN",axis=1)
    Y_train1 = Y_train.iloc[:,1]
    Y_train1.dropna(inplace =True)
    Y_train1 = Y_train1.values
    Y_train1 = (Y_train1 - np.mean(Y_train1))/np.std(Y_train1)
    
    #Standardize X_train
    X_train1 = scaler.fit_transform(X_train1)
    
    #HMLasso
    lasso = hml.HMLasso(mu)
    lasso.fit(X_train1, Y_train1)
    
    #Selection of variables
    coefficients = np.abs(lasso.beta_opt.copy())

    var_to_keep = coefficients < 10**(limit)
    var_to_keep = np.insert(var_to_keep,0,False)
    
    entire_data = liste[0]
    selected = entire_data[entire_data.columns[~var_to_keep]]
    
    return selected

function to impute missing data created when merging by mean but without touching Na values already there before the merge.

In [19]:
def Na_management(df1, df2, index):
    
    merged = df1.merge(df2, how='outer', on = index)
    
    df1_index = df1.set_index(index)
    df2_index = df2.set_index(index)
    
    merged = merged.fillna(merged.mean())
    merged = merged.set_index(index)
    
    df1_index = df1_index.fillna("NaN")
    merged.update(df1_index)
    
    df2_index = df2_index.fillna("NaN")
    merged.update(df2_index)
    
    merged = merged.replace("NaN",np.nan)
    
    merged = merged.reset_index()
    
    return merged

Function to select variables by HMLasso

In [20]:
def Lasso_selection(X_train, Y_train, mu, limit, reduced):
    
    liste = list_wave(X_train, reduced)
    
    print("HMLasso")
    
    selected = initialize_lasso(liste, Y_train, mu, limit)
    
    scaler = StandardScaler()#(with_std=False)
    hml.ERRORS_HANDLING = "ignore"
    
    for i in range (1,14) :
    
        print("wave",i+1)

        var_to_select = Na_management(selected, liste[i], "HHIDPN")

        Y_train_i = Y_train.iloc[:,[0,i+1]]
        X_Y_train = var_to_select.merge(Y_train_i, how = 'left', on = "HHIDPN")

        Y_train_i = X_Y_train[f"tSNE_GHI{i+1}"]
        X_train_i = X_Y_train.drop([f"tSNE_GHI{i+1}","HHIDPN"], axis =1)

        Y_train_i = Y_train_i.fillna(Y_train_i.mean())
        Y_train_i = Y_train_i.values
        Y_train_i = (Y_train_i - np.mean(Y_train_i))/np.std(Y_train_i)

        X_train_i = scaler.fit_transform(X_train_i)

        lasso = hml.HMLasso(mu)
        lasso.fit(X_train_i, Y_train_i)

        coefficients = np.abs(lasso.beta_opt.copy())

        var_to_keep = coefficients < 10**(limit)
        var_to_keep = np.insert(var_to_keep,0,False)

        entire_data = var_to_select
        selected = entire_data[entire_data.columns[~var_to_keep]] 
    
    return selected

In [21]:
selected =Lasso_selection(X_train, Y_train, mu = 200, limit = -14, reduced=False)
selected

HMLasso
wave 1
wave 2
[Warning] Sigma_opt is not PSD, its minimum eigenvalue is -4.5464001906118986e-06. Error handled by adding 4.5464001906118986e-06 to each eigenvalue.
wave 3
[Warning] Sigma_opt is not PSD, its minimum eigenvalue is -6.804841462278468e-06. Error handled by adding 6.804841462278468e-06 to each eigenvalue.
wave 4
[Warning] Sigma_opt is not PSD, its minimum eigenvalue is -5.4432311370396063e-05. Error handled by adding 5.4432311370396063e-05 to each eigenvalue.
wave 5
[Warning] Sigma_opt is not PSD, its minimum eigenvalue is -3.871227124529014e-06. Error handled by adding 3.871227124529014e-06 to each eigenvalue.
wave 6
[Warning] Sigma_opt is not PSD, its minimum eigenvalue is -4.204490209059371e-06. Error handled by adding 4.204490209059371e-06 to each eigenvalue.
wave 7
[Warning] Sigma_opt is not PSD, its minimum eigenvalue is -3.5785352014151535e-05. Error handled by adding 3.5785352014151535e-05 to each eigenvalue.
wave 8
[Warning] Sigma_opt is not PSD, its minimu

C:\Users\Kilian\anaconda3\lib\site-packages\sklearn\utils\extmath.py:985: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\Kilian\anaconda3\lib\site-packages\sklearn\utils\extmath.py:990: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
C:\Users\Kilian\anaconda3\lib\site-packages\sklearn\utils\extmath.py:1020: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction ** 2 / new_sample_count


[Warning] Sigma_opt is not PSD, its minimum eigenvalue is -6.040952078111953e-05. Error handled by adding 6.040952078111953e-05 to each eigenvalue.


,HHIDPN,R1MLEN,R1MCURLN,R1MLENM,R1MNEV,H1ANYFIN,H1ANYFAM,R1FAMR,R1FINR,H1HHRESP,...,H14HHID,R14IWBEG,R14IWEND,R14IADL5H_5,R14IADL5H_4,R14IADL5H_3,genetic_4_NEUROT_SSGAC16,genetic_4_HTN_COGENT17,RABPLACE_11.0,RAVETRN
0,47057010,26.500000,NaN,0.000000,0.000000,1.000000,1.000000,1.000000,1.00000,1.000000,...,470570.0,21319.0,21319.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0
1,64039040,0.000000,NaN,0.000000,1.000000,1.000000,1.000000,1.000000,0.00000,2.000000,...,640390.0,21442.0,21442.0,0.0,0.0,0.0,NaN,NaN,1.0,0.0
2,15204020,31.900000,31.900000,0.000000,0.000000,1.000000,1.000000,1.000000,0.00000,2.000000,...,152040.0,21350.0,21350.0,0.0,0.0,0.0,1.41215,0.38921,0.0,0.0
3,39784010,32.000000,32.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.00000,2.000000,...,397840.0,21350.0,21350.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0
4,54540020,41.300000,41.300000,0.000000,0.000000,1.000000,1.000000,1.000000,0.00000,2.000000,...,545400.0,21411.0,21411.0,0.0,0.0,0.0,0.34175,-1.33294,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1366,544603011,25.908889,26.350769,0.022222,0.034921,0.996825,0.993651,0.660317,0.55873,1.812698,...,5446030.0,21684.0,21715.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0
1367,526039011,25.908889,26.350769,0.022222,0.034921,0.996825,0.993651,0.660317,0.55873,1.812698,...,5260391.0,21684.0,21684.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0
1368,527618022,25.908889,26.350769,0.022222,0.034921,0.996825,0.993651,0.660317,0.55873,1.812698,...,5276182.0,21564.0,21564.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0
1369,545506011,25.908889,26.350769,0.022222,0.034921,0.996825,0.993651,0.660317,0.55873,1.812698,...,5455060.0,21684.0,21684.0,0.0,0.0,0.0,NaN,NaN,0.0,1.0


### Within estimator

There two types of missing values, the "one-time" missing values when someone didn't awnser a question during the interview or so and the missing values when someone wasn't interviewed at all during a wave.


For the first type, we impute those missing values with the mean of the column (Nan).
(Possibility to work on another imputation method).

Then for the individuals who weren't interviewed during a wave, we replace the missing value with the temporal mean of the variable over time (NanNan)


Finally, we compute (A faire en latex) X_vague_ti = X_ti - temporal_mean(X_ti)

In [103]:
def creation_data_within (selected):
    
    #For the "one-time" missing values imputation by mean
    X_train_within = selected.fillna(selected.mean())
    
    ###For people who weren't interviewed
    # We start by adding the INWw columns to know if the individual was interviewed during the wave w
    X_train_within = X_train_within.merge(X_train[["HHIDPN"]+["INW"+str(i) for i in range(1,15)]], how ="left", on="HHIDPN")
    
    #We recover the missing values for people who weren't interviewed during the wave w
    X_train_within = recover_missing(X_train_within)
    
    # Creation of the data set for within regression.
    #X_train_within = data_set_within (X_train_within)
    temporal_variables_within = temporal_variables(X_train_within, False)[0]
    
    return X_train_within

This function return a dataset containing only variables concerned by the wave.

In [23]:
import re
import pickle

def get_wave(data, wave):
  """
  This function returns a smaller dataset summarizing all data for the given wave.

  Note that it also returns columns that are not relative to any wave (for instance, 'HHIDPN')
  """

  assert wave in range(1, 15)

  regex = re.compile("[0-9]+")
  wave_columns = [col for col in data.columns if (len(regex.findall(col)) == 0 or regex.findall(col)[0] == str(wave))]
  wave_data = data[wave_columns]

  return wave_data

Function to recover the missing values for people who weren't interviewed during the wave w

In [44]:
def recover_missing(X_train_within):
### Problem here
    X_train_within_index = X_train_within.set_index("HHIDPN")
    wave_1 = get_wave(X_train_within_index,1)
    wave_1.loc[wave_1["INW1"] == 0] = np.nan
    wave_1["INW1"].fillna(0, inplace =True)
    Tempo = wave_1

    for i in range(2,15):
        wave_i = get_wave(X_train_within_index, i)
        wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
        wave_i["INWw".replace('w', str(i))].fillna(0, inplace =True)
        Tempo = Tempo.merge(wave_i, how= "left", on = "HHIDPN")
    
    X_train_within = Tempo.reset_index()

    return X_train_within

Get a dataframe to know which variables are in X_train_within

Function to compute (A faire en latex) X_vague_ti = X_ti - temporal_mean(X_ti)

It creates columns containing the temporal mean of a temporal variables and then replaces the Nan values (when people weren't interviewed) by this mean. Finally it creates the dataset  X_vague_ti.

In [94]:
def data_set_within (X_train_within):
    temporal_variables_within = dataset_temporal_variables(X_train_within, False)[0]
    for col in temporal_variables_within.columns:
        index_wave = temporal_variables_within.index[temporal_variables_within[col]==1].tolist()
        names_waves = [col.replace('w', str(i+1)) for i in index_wave]
        # (~X_train_within[names_waves].isna()).sum(axis=1) = number of non missing values
        X_train_within[col+"_MEAN"] = X_train_within[names_waves].sum(axis=1)/(~X_train_within[names_waves].isna()).sum(axis=1)
        for x in names_waves:
            # Imputing the missing values by the temporal mean
            new_col = X_train_within[x].fillna(X_train_within[col+"_MEAN"], inplace =True)
            X_train_within[x] = new_col            
            #Creating the new data for within regression X_vague
            X_train_within[x] = X_train_within[x] - X_train_within[col+"_MEAN"]
    return X_train_within

In [126]:
X_train_within = selected.fillna(selected.mean())

In [127]:
# We start by adding the INWw columns to know if the individual was interviewed during the wave w
X_train_within = X_train_within.merge(X_train[["HHIDPN"]+["INW"+str(i) for i in range(1,15)]], how ="left", on="HHIDPN")

In [128]:
X_train_within_index = X_train_within.set_index("HHIDPN")
wave_1 = get_wave(X_train_within_index,1)
wave_1.loc[wave_1["INW1"] == 0] = np.nan
wave_1["INW1"].fillna(0)
Tempo = wave_1

for i in range(2,15):
    wave_i = get_wave(X_train_within_index, i)
    wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
    wave_i["INWw".replace('w', str(i))].fillna(0)
    Tempo = Tempo.merge(wave_i, how= "left", on = "HHIDPN")
    
X_train_within = Tempo.reset_index()

C:\Users\Kilian\AppData\Local\Temp\ipykernel_3292\3146717300.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_3292\3146717300.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_3292\3146717300.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [129]:
temporal_variables_within = dataset_temporal_variables(X_train_within, False)[0]

for col in temporal_variables_within.columns:
    index_wave = temporal_variables_within.index[temporal_variables_within[col]==1].tolist()
    names_waves = [col.replace('w', str(i+1)) for i in index_wave]
    # (~X_train_within[names_waves].isna()).sum(axis=1) = number of non missing values
    X_train_within[col+"_MEAN"] = X_train_within[names_waves].sum(axis=1)/(~X_train_within[names_waves].isna()).sum(axis=1)
    for x in names_waves:
        # Imputing the missing values by the temporal mean
        new_col = X_train_within[x].fillna(X_train_within[col+"_MEAN"])
        X_train_within[x] = new_col
        #Creating the new data for within regression X_vague
        X_train_within[x] = X_train_within[x] - X_train_within[col+"_MEAN"]

In [130]:
#Still Nan values in intemporal variables
X_train_within = X_train_within.fillna(X_train_within.mean())

Now we do the same thing to Y_train but no need to impute the Nan values since the only outcome is tSNE_GHI14 (no Nan).

In [131]:
Y_train_within = Y_train.copy()

In [132]:
tSNE_GHI = [f"tSNE_GHI{w}" for w in range(1,15)]

Y_train_within["tSNE_GHIw_MEAN"] = Y_train_within[tSNE_GHI].sum(axis=1)/(~Y_train_within[tSNE_GHI].isna()).sum(axis=1)
Y_train_within["tSNE_GHI14_within"] = Y_train_within["tSNE_GHI14"] - Y_train_within["tSNE_GHIw_MEAN"]

We can now proceed to the regression

In [133]:
data_regression = X_train_within.merge(Y_train_within[["HHIDPN","tSNE_GHI14_within"]], on = "HHIDPN")
Y_regression = data_regression["tSNE_GHI14_within"]
list_to_drop = ["HHIDPN","tSNE_GHI14_within"]+["INW"+str(i) for i in range(1,15)]+[col+"_MEAN" for col in temporal_variables_within.columns]
X_regression = data_regression.drop(list_to_drop,axis=1)

In [134]:
from sklearn.linear_model import LinearRegression

modeleReg=LinearRegression()

modeleReg.fit(X_regression,Y_regression)

LinearRegression()

In [136]:
print(modeleReg.intercept_)
print(modeleReg.coef_)

#calcul du R²
modeleReg.score(X_regression,Y_regression)

0.5349960075470737
[ 3.31457109e+02  1.60161892e+10  2.15141725e+10  2.50502246e+10
  8.21665937e+09 -2.40215262e+09 -1.70383192e+09  8.45570682e+09
 -1.25665826e+09  3.43841941e+09  6.60150261e+07  1.58178500e+10
 -2.74678943e+11 -5.74144140e+01  3.27798836e+02  1.60161892e+10
  6.72283321e+09 -1.40203653e+10  8.21665908e+09 -2.40215265e+09
 -1.70383195e+09  8.45570673e+09 -1.25665826e+09  3.43841936e+09
  6.60150368e+07  1.58178500e+10 -2.74678943e+11 -2.85591043e+01
  3.33226066e+02  1.60161892e+10 -2.91829694e+07  2.14857374e+10
  8.21665924e+09 -1.70383191e+09  8.45570683e+09 -1.25665821e+09
  3.43841936e+09  6.60150579e+07  1.58178500e+10 -2.74678943e+11
  2.06760376e+07  7.78617094e+01  3.33145467e+02  1.60161892e+10
 -1.50663040e+09 -1.14270834e+10  8.21665906e+09 -2.40215274e+09
 -1.70383192e+09  8.45570684e+09 -1.25665818e+09  3.43841930e+09
  6.60150247e+07  1.58178500e+10 -2.74678943e+11  2.22824665e+07
  2.06760399e+07  3.31026553e+00  6.05580546e-01  2.89362468e+01
  3.32

0.24012328908697889

In [74]:
Y_train_within

,HHIDPN,tSNE_GHI1,tSNE_GHI2,tSNE_GHI3,tSNE_GHI4,tSNE_GHI5,tSNE_GHI6,tSNE_GHI7,tSNE_GHI8,tSNE_GHI9,tSNE_GHI10,tSNE_GHI11,tSNE_GHI12,tSNE_GHI13,tSNE_GHI14,tSNE_GHIw_MEAN,tSNE_GHI14_within
35141,540304010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.804930,61.886147,63.345539,-1.459392
30248,502218020,NaN,NaN,NaN,NaN,NaN,NaN,48.852474,37.104267,56.103394,50.725037,-6.405303,-39.898094,6.425584,-15.314579,17.199097,-32.513676
40563,907690020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.819390,48.003716,-14.311967,-45.567310,27.308905,11.650547,15.658358
6954,47057010,-57.655020,24.923943,-19.287235,-11.867963,-45.765656,-51.708080,66.945540,19.890480,-44.744183,-76.391785,-15.420312,-26.248714,-20.455055,-1.869290,-18.546666,16.677376
26950,213031010,NaN,NaN,NaN,-53.555614,-24.401985,-92.002350,-27.655800,59.701550,66.647790,99.778190,3.915255,19.751297,6.718432,-20.618025,3.479885,-24.097910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32542,525847020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-55.090855,-20.056421,-20.870111,57.827724,65.454980,5.453063,60.001917
28487,500892010,NaN,NaN,NaN,NaN,NaN,NaN,-16.402885,-101.593440,-18.303556,-67.452095,-77.100980,-65.834885,-66.255850,-64.370340,-59.664254,-4.706086
15978,177715010,NaN,NaN,NaN,9.898155,-41.924114,-79.426710,-41.448692,-55.587166,-58.608147,-52.567307,-45.682487,95.338830,-56.426400,94.130300,-21.118522,115.248822
3200,24400020,6.360288,60.418080,111.114470,23.114595,0.542709,11.005449,7.788031,70.938190,-12.596005,74.012405,81.418304,58.977135,70.751150,20.058348,41.707368,-21.649020
